In [1]:
import torch
import os
import mlflow
import numpy as np
import matplotlib as mpl
from ultralytics import YOLO
import ultralytics
import pycocotools.coco
import pycocowriter.coco2yolo
import requests
import sys
from hierarchical_yolo.hierarchical_detection import *
from hierarchical_yolo.deep7_model import *

In [2]:
'''
deep7 categories:

  0: Kalekale
  1: Opakapaka
  2: "Hapu\u02BBupu\u02BBu"
  3: Gindai
  4: Other or Can't Tell
  5: Ehu
  6: Lehi
  7: Onaga
  8: Snapper
  9: Grouper
  10: Object
'''

deep7_hierarchy = {
    0: 8,
    1: 8,
    2: 9,
    3: 8,
    5: 8,
    6: 8,
    7: 8,
    4: 10,
    8: 10,
    9: 10
}

In [3]:
'''
deep7 categories:

  0: Kalekale
  1: Opakapaka
  2: "Hapu\u02BBupu\u02BBu"
  3: Gindai
  4: Fish
  5: Ehu
  6: Lehi
  7: Onaga
'''

deep7_hierarchy = {
    0: 4,
    1: 4,
    2: 4,
    3: 4,
    5: 4,
    6: 4,
    7: 4
}

In [4]:
from ultralytics import settings

# View all settings
print(settings)

JSONDict("/home/noaa_brown/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/noaa_brown/datasets",
  "weights_dir": "/home/noaa_brown/hierarchical_yolo/weights",
  "runs_dir": "/home/noaa_brown/hierarchical_yolo/runs",
  "uuid": "2768c477e7938b5779aff75aed83103f60cf5680dbe7372dfa0b27c233b7d105",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


In [5]:
devices = list(range(torch.cuda.device_count()))
for i in devices:
    print(torch.cuda.get_device_properties(i).name)

NVIDIA GeForce GTX 1080 Ti
NVIDIA GeForce GTX 1080 Ti
NVIDIA GeForce GTX 1080 Ti
NVIDIA GeForce GTX 1080 Ti
NVIDIA GeForce GTX 1080 Ti
NVIDIA GeForce GTX 1080 Ti
NVIDIA GeForce GTX 1080 Ti


In [6]:
DATA = '../data'
UPLOAD_URL = 'https://storage.googleapis.com/nmfs_odp_hq/nodd_tools/datasets/oceaneyes/annotation_number_balanced_sample/annotations.json'
DOWNLOAD_PATH = os.path.join(DATA, 'download')
COCO_PATH = os.path.join(DOWNLOAD_PATH, 'annotations.json')
YOLO_PATH = os.path.join(DOWNLOAD_PATH, 'yolo_training_data')
IMAGES_PATH = os.path.join(YOLO_PATH, 'annotations', 'images')
os.makedirs(YOLO_PATH, exist_ok=True)
os.makedirs(DOWNLOAD_PATH, exist_ok=True)

In [7]:
uploaded_coco_file = requests.get(UPLOAD_URL)
with open(COCO_PATH, 'wb') as f:
    f.write(uploaded_coco_file.content)
    coco = pycocotools.coco.COCO(COCO_PATH)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [8]:
coco.info()

year: 2024
version: 0.1
description: https://www.zooniverse.org/projects/benjamin-dot-richards/oceaneyes/about/research
contributor: None
url: None
date_created: 2025-02-06T20:56:54.886937+00:00


In [9]:
pycocowriter.coco2yolo.coco2yolo(DOWNLOAD_PATH, YOLO_PATH)

Annotations /home/noaa_brown/hierarchical_yolo/data/download/annotations.json: 100%|██████████| 200/200 [00:00<00:00, 1072.98it/s]

COCO data converted successfully.
Results saved to /home/noaa_brown/hierarchical_yolo/notebooks/coco_converted


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
downloaded 0/200 images (t=0.0s)
downloaded 1/200 images (t=0.0s)
downloaded 2/200 images (t=0.0s)
downloaded 3/200 images (t=0.0s)
downloaded 4/200 images (t=0.0s)
downloaded 5/200 images (t=0.0s)
downloaded 6/200 images (t=0.0s)
downloaded 7/200 images (t=0.0s)
downloaded 8/200 images (t=0.0s)
downloaded 9/200 images (t=0.0s)
downloaded 10/200 images (t=0.0s)
downloaded 11/200 images (t=0.0s)
downloaded 12/200 images (t=0.0s)
downloaded 13/200 images (t=0.0s)
downloaded 14/200 images (t=0.0s)
downloaded 15/200 images (t=0.0s)
downloaded 16/200 images (t=0.0s)
downloaded 17/200 images (t=0.0s)
downloaded 18/200 images (t=0.0s)
downloaded 19/200 images (t=0.0s)
downloaded 20/200 images (t=0.0s)
downloaded 21/200 images (t=0.0s)
downloaded 22/200 images (t=0.0s)
downloaded 23/200 images (t=0.0s)
downloaded 24/200 images (t=

In [10]:
# TODO: the yolov8.yaml model file downloaded from Ultralytics needs manual editing for the number of classes
YOLO_YAML = os.path.join(DATA, 'yolov8.yaml')
YOLO_BASE_MODEL = os.path.join(DATA, 'yolov8n.pt')
YOLO_TRAIN_YAML = os.path.join(YOLO_PATH, 'train.yaml')

In [11]:
mlflow.autolog()

In [12]:
model = YOLO(YOLO_YAML).load(YOLO_BASE_MODEL)  # build a new model from scratch

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Transferred 319/355 items from pretrained weights


In [13]:
results = model.train(
    data=YOLO_TRAIN_YAML, 
    epochs=5, imgsz=640, 
    device=devices, 
    batch=max(len(devices), 1),
    trainer=Deep7HierarchicalDetectionTrainer
)

New https://pypi.org/project/ultralytics/8.3.111 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.104 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
                                                       CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
                                                       CUDA:2 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
                                                       CUDA:3 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
                                                       CUDA:4 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
                                                       CUDA:5 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
                                                       CUDA:6 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: task=detect, mode=train, model=../data/yolov8.yaml, data=../data/download/yolo_training_data/train.yaml, epochs=5, time=None, patience=100, batch=7, imgsz=640, save=True, save_

train: Scanning /home/noaa_brown/hierarchical_yolo/data/download/yolo_training_data/annotations/labels.cache... 200 images, 0 backgrounds, 1 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


train: WARNING ⚠️ /home/noaa_brown/hierarchical_yolo/data/download/yolo_training_data/annotations/images/20190913_201709_20190913.203153.002.010131.jpg: ignoring corrupt image/label: negative label values [-0.00068681]


val: Scanning /home/noaa_brown/hierarchical_yolo/data/download/yolo_training_data/annotations/labels.cache... 200 images, 0 backgrounds, 1 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


train: WARNING ⚠️ /home/noaa_brown/hierarchical_yolo/data/download/yolo_training_data/annotations/images/20190913_201709_20190913.203153.002.010131.jpg: ignoring corrupt image/label: negative label values [-0.00068681]
Plotting labels to /home/noaa_brown/hierarchical_yolo/runs/detect/train72/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0004921875), 63 bias(decay=0.0)
MLflow: logging run_id(1b7bfc948e84446ea170b8317f657aa6) to /home/noaa_brown/hierarchical_yolo/runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri /home/noaa_brown/hierarchical_yolo/runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 14 dataloader workers
Logging results to /home/noaa_brown/hierarchical_yolo/runs/dete

  0%|          | 0/29 [00:00<?, ?it/s]

targets
torch.Size([1, 8400, 8])
torch.Size([1, 8400])
tensor([[0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')
torch.Size([8, 2])
torch.Size([1, 8400, 2])
torch.Size([1, 8400, 2])
[torch.Size([1, 8400, 2]), torch.Size([1, 8400, 2]), torch.Size([1, 8400, 2])]
torch.Size([1, 8400, 2])
torch.Size([1, 8400, 2])
torch.Size([1, 8400, 2])


  0%|          | 0/29 [00:00<?, ?it/s]
[rank0]: Traceback (most recent call last):
[rank0]:   File "/home/noaa_brown/.config/Ultralytics/DDP/_temp_8to57xks134598790317360.py", line 13, in <module>
[rank0]:     results = trainer.train()
[rank0]:               ^^^^^^^^^^^^^^^
[rank0]:   File "/home/noaa_brown/hierarchical_yolo/.venv/lib/python3.12/site-packages/ultralytics/engine/trainer.py", line 211, in train
[rank0]:     self._do_train(world_size)
[rank0]:   File "/home/noaa_brown/hierarchical_yolo/.venv/lib/python3.12/site-packages/ultralytics/engine/trainer.py", line 385, in _do_train
[rank0]:     loss, self.loss_items = self.model(batch)
[rank0]:                             ^^^^^^^^^^^^^^^^^
[rank0]:   File "/home/noaa_brown/hierarchical_yolo/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
[rank0]:     return self._call_impl(*args, **kwargs)
[rank0]:            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
[rank0]:   File "/home/noaa_brown/hierar

CalledProcessError: Command '['/home/noaa_brown/hierarchical_yolo/.venv/bin/python3', '-m', 'torch.distributed.run', '--nproc_per_node', '7', '--master_port', '60379', '/home/noaa_brown/.config/Ultralytics/DDP/_temp_8to57xks134598790317360.py']' returned non-zero exit status 1.

In [ ]:
random_img = os.path.join(IMAGES_PATH, str(np.random.choice(os.listdir(IMAGES_PATH))))
random_img

In [ ]:
MODEL_LOCATION = os.path.join('..','runs','detect')
trained_models = os.listdir(os.path.join('..','runs','detect'))
model_numbers = map(lambda x: int(x[len('train'):]) if len(x) > len('train') else 0, trained_models)
latest_model = 'train' + str(max(model_numbers))

In [ ]:
trained_model = YOLO(
    YOLO_YAML
).load(os.path.join(MODEL_LOCATION, latest_model, 'weights', 'best.pt'))

In [ ]:
predictions = trained_model.predict(random_img, verbose=False, device=devices[:1], stream=True)

In [ ]:
prediction = next(predictions)

In [ ]:
prediction

In [ ]:
prediction.boxes

In [ ]:
prediction.save('pickles.jpg')
from IPython.display import Image
Image(filename='pickles.jpg') 